In [1]:
pip install msgpack


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import importlib.util
import json
import sys
from pathlib import Path

ROOT = Path("/home/bogao/sim-arena")
sys.path.insert(0, str(ROOT))

spec = importlib.util.spec_from_file_location("sk_action", ROOT / "sk-action.py")
sk_action = importlib.util.module_from_spec(spec)
assert spec.loader is not None
spec.loader.exec_module(sk_action)

make_action = sk_action.make_action
from env.actions.trace_io import json_to_msgpack, load_trace



In [2]:
TRACE_JSON = ROOT / "action_demo" / "trace-0001.json"
TRACE_PATH = Path(json_to_msgpack(str(TRACE_JSON)))
baseline_trace = load_trace(str(TRACE_PATH))

first_event = baseline_trace["events"][0]
first_deploy = first_event["applied_objs"][0]
print("starting replicas:", first_deploy["spec"]["replicas"])
print("starting cpu:", first_deploy["spec"]["template"]["spec"]["containers"][0]["resources"]["requests"]["cpu"])  # noqa: E501
print("starting memory:", first_deploy["spec"]["template"]["spec"]["containers"][0]["resources"]["requests"]["memory"])  # noqa: E501



starting replicas: 2
starting cpu: 500m
starting memory: 512Mi


In [3]:
def apply_action(op: str, **kwargs):
    action = make_action(op, deploy="web", **kwargs)
    next_trace, reward, done, info = action(baseline_trace)
    assert not done
    return next_trace, reward, info



In [4]:
cpu_trace, cpu_reward, cpu_info = apply_action("bump_cpu_small", step="250m")

cpu_requests = cpu_trace["events"][0]["applied_objs"][0]["spec"]["template"]["spec"]["containers"][0]["resources"]["requests"]
print("updated cpu:", cpu_requests["cpu"], "reward:", cpu_reward)

assert cpu_info["changed"] is True
assert cpu_requests["cpu"] == "750m"
assert any(entry["path"][-1] == "cpu" for entry in cpu_info["diff"])

print(json.dumps(cpu_info, indent=2))



updated cpu: 750m reward: 1.0
{
  "changed": true,
  "op": "bump_cpu_small",
  "deploy": "web",
  "diff": [
    {
      "path": [
        "events",
        0,
        "applied_objs",
        0,
        "spec",
        "template",
        "spec",
        "containers",
        0,
        "resources",
        "requests",
        "cpu"
      ],
      "before": "500m",
      "after": "750m"
    }
  ]
}


In [5]:
mem_trace, mem_reward, mem_info = apply_action("bump_mem_small", step="512Mi")

mem_requests = mem_trace["events"][0]["applied_objs"][0]["spec"]["template"]["spec"]["containers"][0]["resources"]["requests"]
print("updated memory:", mem_requests["memory"], "reward:", mem_reward)

assert mem_info["changed"] is True
assert mem_requests["memory"] == "1024Mi"
assert any(entry["path"][-1] == "memory" for entry in mem_info["diff"])

print(json.dumps(mem_info, indent=2))



updated memory: 1024Mi reward: 1.0
{
  "changed": true,
  "op": "bump_mem_small",
  "deploy": "web",
  "diff": [
    {
      "path": [
        "events",
        0,
        "applied_objs",
        0,
        "spec",
        "template",
        "spec",
        "containers",
        0,
        "resources",
        "requests",
        "memory"
      ],
      "before": "512Mi",
      "after": "1024Mi"
    }
  ]
}


In [6]:
mem_trace


{'version': 1,
 'metadata': {'description': 'Synthetic trace for local development'},
 'events': [{'ts': 1730390400,
   'applied_objs': [{'apiVersion': 'apps/v1',
     'kind': 'Deployment',
     'metadata': {'name': 'web', 'namespace': 'default'},
     'spec': {'replicas': 2,
      'selector': {'matchLabels': {'app': 'web'}},
      'template': {'metadata': {'labels': {'app': 'web'}},
       'spec': {'containers': [{'name': 'web',
          'image': 'ghcr.io/example/web:1.0',
          'resources': {'requests': {'cpu': '500m',
            'memory': '1024Mi'}}}]}}}}],
   'deleted_objs': []}]}

In [7]:
rep_trace, rep_reward, rep_info = apply_action("scale_up_replicas", delta=2)

replicas = rep_trace["events"][0]["applied_objs"][0]["spec"]["replicas"]
print("updated replicas:", replicas, "reward:", rep_reward)

assert rep_info["changed"] is True
assert replicas == 4
assert any(entry["path"][-1] == "replicas" for entry in rep_info["diff"])

print(json.dumps(rep_info, indent=2))



updated replicas: 4 reward: 1.0
{
  "changed": true,
  "op": "scale_up_replicas",
  "deploy": "web",
  "diff": [
    {
      "path": [
        "events",
        0,
        "applied_objs",
        0,
        "spec",
        "replicas"
      ],
      "before": 2,
      "after": 4
    }
  ]
}
